# Temporal derivatives

We'll compare and contrast methods from
[Tripp](http://compneuro.uwaterloo.ca/publications/tripp2010.html)
and [Voelker](https://github.com/ctn-waterloo/summerschool2015/tree/master/tutorials/temprep).

In [ ]:
# Common imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import nengo
import nengo.utils.numpy as npext

# Some plotting niceties
plt.rc('figure', figsize=(10, 6))
sns.set_style('white')
sns.set_style('ticks')

In [ ]:
def test_deriv(deriv_func, deriv_args, dims, ramp=True, t=1.0, dt=0.001):
    with nengo.Network() as net:
        if ramp:
            timesteps = int(t / dt)
            ramp = np.concatenate([np.linspace(0, 1, timesteps/2), np.linspace(1, 0, timesteps/2)])
            proc = lambda time: ramp[int(time / dt) % ramp.size]
        else:
            proc = nengo.processes.BrownNoise()
        inp = nengo.Node(size_in=dims)
        for i in range(dims):
            nengo.Connection(nengo.Node(output=proc), inp[i])
        ea = nengo.networks.EnsembleArray(n_neurons=40, n_ensembles=dims)
        nengo.Connection(inp, ea.input)
        deriv = deriv_func(dimensions=dims, **deriv_args)
        nengo.Connection(ea.output, deriv.input)
        in_probe = nengo.Probe(inp, synapse=None)
        out_probe = nengo.Probe(deriv.output, synapse=0.01)
    print("%d neurons" % (sum(e.n_neurons for e in net.all_ensembles)))
    sim = nengo.Simulator(net)
    sim.run(t)

    inp = sim.data[in_probe]
    plt.subplot(2, 1, 1)
    plt.plot(sim.trange(), inp)
    plt.ylabel("Input")
    plt.subplot(2, 1, 2)
    plt.plot(sim.trange(), sim.data[out_probe])

### Tripp 1: Feedforward, intermediate population

In [ ]:
def Derivative(dimensions, n_neurons=100, tau=0.01, net=None):
    if net is None:
        net = nengo.Network(label="Derivative")

    with net:
        net.input = nengo.Ensemble(n_neurons * dimensions, dimensions)
        net.intermediate = nengo.Ensemble(n_neurons * dimensions, dimensions)
        net.output = nengo.Ensemble(n_neurons * dimensions, dimensions)
        nengo.Connection(net.input, net.intermediate, synapse=tau)
        nengo.Connection(net.input, net.output, synapse=tau, transform=1/tau)
        nengo.Connection(net.intermediate, net.output, synapse=tau, transform=-1/tau)
    return net

test_deriv(Derivative, {}, dims=2)

### Tripp 2: Feedforward, different synapses

In [ ]:
def Derivative(dimensions, n_neurons=100,
               tau_fast=0.01, tau_slow=0.1, net=None):
    if net is None:
        net = nengo.Network(label="Derivative")
    
    with net:
        tau_diff = tau_slow - tau_fast
        net.input = nengo.Ensemble(n_neurons * dimensions, dimensions)
        net.output = nengo.Ensemble(n_neurons * dimensions, dimensions)
        nengo.Connection(net.input, net.output, synapse=tau_fast, transform=1/tau_diff)
        nengo.Connection(net.input, net.output, synapse=tau_slow, transform=-1/tau_diff)
    return net

test_deriv(Derivative, {}, dims=2)

### Tripp 3: Feedback approximating 2

In [ ]:
def adjust_abc(old_a, old_b, old_c, dimensions):
    degree = old_a.shape[0]
    a = np.zeros((degree * dimensions, degree * dimensions))
    b = np.zeros((degree * dimensions, dimensions))
    c = np.zeros((dimensions, degree * dimensions))

    # Replicate the existing system in the right blocks
    for dim in range(dimensions):
        blk = slice(dim * degree, (dim+1) * degree)
        a[blk, blk] = old_a
        b[blk, dim] = old_b[:, 0]
        c[dim, blk] = old_c
    return a, b, c

def FeedbackDerivative(dimensions, a, b, c,
                       n_neurons=100, tau=0.005, net=None):
    if net is None:
        net = nengo.Network(label="Derivative")
    
    tau_a = np.identity(a.shape[0]) + a * tau
    tau_b = b * tau
    tau_a, tau_b, c = adjust_abc(tau_a, tau_b, c, dimensions)

    with net:
        net.input = nengo.Ensemble(n_neurons * dimensions, dimensions)
        net.output = nengo.Ensemble(n_neurons * dimensions, dimensions)
        net.diff = nengo.Ensemble(n_neurons * dimensions * 2, dimensions * 2)
        nengo.Connection(net.input, net.diff, synapse=tau, transform=tau_b)
        nengo.Connection(net.diff, net.diff, synapse=tau, transform=tau_a)
        nengo.Connection(net.diff, net.output, transform=c)
    return net


def Derivative(dimensions, n_neurons=100, tau=0.01, net=None):
    a = np.array([[-5, -7.5], [3.3333, -15]])
    b = np.array([[10], [20]])
    c = np.array([[10, 0]])
    return FeedbackDerivative(dimensions, a, b, c, n_neurons, tau, net)

test_deriv(Derivative, {}, dims=2)

### Tripp 4: Butterworth filter

In [ ]:
def Derivative(dimensions, n_neurons=100, tau=0.01, net=None):
    a = np.array([[-8.8858, 19.9931], [-3.9492, -8.8858]])
    b = np.array([[27.4892], [-12.2174]])
    c = np.array([[5.7446, 0]])
    return FeedbackDerivative(dimensions, a, b, c, n_neurons, tau, net)

test_deriv(Derivative, {}, dims=2)

### Voelker

In [ ]:
from nengo.utils.filter_design import zpk2ss, tf2ss, cont2discrete
from scipy.linalg import solve_lyapunov
from scipy.misc import factorial, pade


class LTI(object):
    """Methods for dealing with LTI filters in Nengo.

    Adapted from Aaron Voelker's delay notebook at
    summerschool2015/tutorials/temprep/delay.ipynb
    """
    def __init__(self, a, b, c, d):
        self.a = np.array(a)
        self.b = np.array(b)
        self.c = np.array(c)
        self.d = np.array(d)

    @property
    def abcd(self):
        return (self.a, self.b, self.c, self.d)

    @classmethod
    def from_synapse(cls, synapse):
        """Instantiate class from a Nengo synapse."""
        if not hasattr(synapse, 'num') or not hasattr(synapse, 'den'):
            raise ValueError("Must be a linear filter with 'num' and 'den'")
        return cls(*tf2ss(synapse.num, synapse.den))

    @classmethod
    def from_tf(cls, num, den):
        """Instantiate class from a transfer function."""
        return cls(*tf2ss(num, den))

    @classmethod
    def from_zpk(cls, z, p, k):
        """Instantiate class from a zero-pole-gain representation."""
        return cls(*zpk2ss(z, p, k))

    def copy(self):
        return LTI(*self.abcd)

    def scale_to(self, radii=1.0):
        """Scales the system to give an effective radius of r to x."""
        r = np.asarray(radii, dtype=np.float64)
        if r.ndim > 1:
            raise ValueError(
                "radii (%s) must be a 1-D array or scalar" % radii)
        elif r.ndim == 0:
            r = np.ones(len(self.a)) * r
        self.a = self.a / r[:, None] * r
        self.b /= r[:, None]
        self.c *= r

    def ab_norm(self):
        """Returns H2-norm of each component of x in the state-space.

        Equivalently, this is the H2-norm of each component of (A, B, I, 0).
        This gives the power of each component of x in response to white-noise
        input with uniform power.

        Useful for setting the radius of an ensemble array with continuous
        dynamics (A, B).
        """
        p = solve_lyapunov(self.a, -np.dot(self.b, self.b.T))  # AP + PA^H = Q
        assert np.allclose(np.dot(self.a, p)
                           + np.dot(p, self.a.T)
                           + np.dot(self.b, self.b.T), 0)
        c = np.eye(len(self.a))
        h2norm = np.dot(c, np.dot(p, c.T))
        # The H2 norm of (A, B, C) is sqrt(tr(CXC^T)), so if we want the norm
        # of each component in the state-space representation, we evaluate
        # this for each elementary vector C separately, which is equivalent
        # to picking out the diagonals
        return np.sqrt(h2norm[np.diag_indices(len(h2norm))])

    def to_sim(self, synapse, dt=0):
        """Maps a state-space LTI to the synaptic dynamics on A and B."""
        if not isinstance(synapse, nengo.Lowpass):
            raise TypeError("synapse (%s) must be Lowpass" % (synapse,))
        if dt == 0:
            a = synapse.tau * self.a + np.eye(len(self.a))
            b = synapse.tau * self.b
        else:
            a, b, c, d, _ = cont2discrete(self.abcd, dt=dt)
            aa = np.exp(-dt / synapse.tau)
            a = 1. / (1 - aa) * (a - aa * np.eye(len(a)))
            b = 1. / (1 - aa) * b
        self.a, self.b, self.c, self.d = a, b, c, d


def exp_delay(p, q, c=1.0):
    """Returns F = p/q such that F(s) = e^(-sc)."""
    i = np.arange(p+q) + 1
    taylor = np.append([1.0], (-c)**i / factorial(i))
    return pade(taylor, q)

In [ ]:
def lti(n_neurons, dimensions, lti_system, synapse=nengo.Lowpass(0.05),
        controlled=False, dt=0.001, radii=None, radius=1.0):
    if radii is None:
        radii = lti_system.ab_norm()
    radii *= radius
    lti_system.scale_to(radii)
    lti_system.to_sim(synapse, dt)

    degree = lti_system.a.shape[0]
    a = np.zeros((degree * dimensions, degree * dimensions))
    b = np.zeros((degree * dimensions, dimensions))
    c = np.zeros((dimensions, degree * dimensions))
    d = np.zeros(dimensions)

    # Replicate the existing system in the right blocks
    for dim in range(dimensions):
        blk = slice(dim * degree, (dim+1) * degree)
        a[blk, blk] = lti_system.a
        b[blk, dim] = lti_system.b[:, 0]
        c[dim, blk] = lti_system.c
        d[dim] = lti_system.d

    size_in = b.shape[1]
    size_state = a.shape[0]
    size_out = c.shape[0]

    inp = nengo.Node(size_in=size_in, label="input")
    out = nengo.Node(size_in=size_out, label="output")
    x = nengo.networks.EnsembleArray(n_neurons, size_state)
    x_in = x.input
    x_out = x.output

    nengo.Connection(x_out, x_in, transform=a, synapse=synapse)
    nengo.Connection(inp, x_in, transform=b, synapse=synapse)
    nengo.Connection(x_out, out, transform=c, synapse=None)
    nengo.Connection(inp, out, transform=d, synapse=None)

    return inp, out


def deconvolution(n_neurons, dimensions, tf, delay, degree=4, **lti_kwargs):
    """Approximate the inverse of a given transfer function using a delay."""
    num, den = [np.poly1d(tf[0]), np.poly1d(tf[1])]
    order = len(den) - len(num)
    if order >= degree:
        raise ValueError("order (%d) must be < degree (%d)"
                         % (order, degree))
    edp, edq = exp_delay(degree - order, degree, delay)
    p, q = np.polymul(edp, den), np.polymul(edq, num)
    inp, out = lti(n_neurons, dimensions, LTI.from_tf(p, q), **lti_kwargs)
    return inp, out, degree


def derivative(n_neurons, dimensions, tau, delay, **deconv_kwargs):
    """Output a signal that is a derivative of the input."""
    return deconvolution(n_neurons, dimensions, ([1], [tau, 0]), delay, **deconv_kwargs)


def Derivative(delay, dimensions,
               n_neurons=100, tau=0.005, tau_highpass=0.05, net=None):
    if net is None:
        net = nengo.Network(label="Derivative")

    with net:
        net.input, net.output, _ = derivative(
            n_neurons, dimensions=dimensions, delay=delay, tau=tau, radius=0.1, degree=2)

        actual_output = nengo.Node(size_in=dimensions)
        nengo.Connection(net.output, actual_output, synapse=tau_highpass)
        net.output = actual_output
    return net

test_deriv(Derivative, {'delay': 0.01}, dims=2)